In [7]:
import pandas as pd
import matplotlib as plt




In [43]:
path = "table pour machine learning test.csv"
Data = pd.read_csv(path,skiprows=1)



Data = Data.drop('!6',axis=1)
values_to_keep =  ["VALN", "ZCB", "EQ", "RE","ILZCB","INFLN"]
Data =  Data[Data['CLASS'].isin(values_to_keep)].reset_index(drop=True)


nb_sim = max(Data["SIMULATION"].unique())
nb_term = max(Data["TERM"].unique())

#Interest rate treatment (Optional): keep ZC with duration

"""Duration = 10 #Optional 
Data =  Data[Data['CLASS'].isin(values_to_keep)]
Data = Data[~((Data['CLASS'] == 'ZCB') & (Data['TERM'] != Duration))]
Data.drop('TERM',axis=1) #Remove term , not needed anymore """

row = Data[(Data["SIMULATION"]==1) & (Data["CLASS"]=="VALN")].loc[:,"2021":].reset_index()
row_2 = Data[(Data["SIMULATION"]==1) & (Data["CLASS"]=="ZCB")].loc[:,"2021":].reset_index()

test = row * row_2

print(row_2)
    







    index      2021      2022      2023      2024      2025      2026  \
0       1  1.005884  0.998936  0.997798  1.004071  1.007830  1.009647   
1       2  1.007947  0.996547  0.996784  1.010954  1.018540  1.021194   
2       3  1.007416  0.994681  0.998025  1.019027  1.031320  1.033663   
3       4  1.005821  0.994444  1.000273  1.027546  1.045110  1.046400   
4       5  1.004211  0.994659  1.003125  1.035596  1.059182  1.059000   
5       6  1.001561  0.995048  1.005888  1.042725  1.073080  1.072048   
6       7  0.997903  0.995047  1.008204  1.048785  1.087369  1.084143   
7       8  0.992988  0.994413  1.009927  1.054657  1.100644  1.095811   
8       9  0.986867  0.993097  1.011885  1.059085  1.113448  1.107674   
9      10  0.979729  0.992000  1.012766  1.062801  1.126422  1.120482   
10     11  0.972695  0.989848  1.013240  1.066582  1.140339  1.134571   
11     12  0.964577  0.987357  1.014018  1.071286  1.155551  1.148543   
12     13  0.956215  0.985258  1.015890  1.077299  

In [9]:

#Equity Like Index : Adjust With dividends and actualize 

result_df = pd.DataFrame()

unique_classes = Data['CLASS'].unique()
nb_sim = max(Data["SIMULATION"].unique())

for equity_classe in unique_classes:

    
    if equity_classe != 'VALN':  
        for i in range(1, nb_sim + 1):

            divs = Data[(Data['MEASURE'] == "RNY_PC") & (Data['SIMULATION'] == i) & (Data['CLASS'] == equity_classe)]
            divs = divs.loc[:, "2021":].squeeze()

            rough_Price = Data[(Data['MEASURE'] == "RET_IDX") & (Data['SIMULATION'] == i) & (Data['CLASS'] == equity_classe)].loc[:, "2021":].squeeze()

            """Adjustment_factor = ((rough_Price - ((divs/100) * rough_Price)) / rough_Price)
            Net_stock_price = rough_Price * Deflator * Adjustment_factor"""
            result_df[(equity_classe, i)] = rough_Price


result_df.columns = pd.MultiIndex.from_tuples(result_df.columns, names=['Equity_Class', 'Simulation'])
df_melted = result_df.reset_index().melt(id_vars='index', var_name=['Equity_Class', 'Simulation'], value_name='Value')
df_melted['Simulation'] = df_melted['Simulation'].astype(int)
df_equity_like = df_melted.pivot(index='Simulation', columns=['Equity_Class', 'index'], values='Value')
df_equity_like.columns = [f"{ec}_{year}" for ec, year in df_equity_like.columns]
df_equity_like.index.name = None
df_equity_like.head()




ValueError: Cannot set a DataFrame with multiple columns to the single column ('ZCB', 1)

In [ ]:
path = "table pour machine learning test.csv"
Data = pd.read_csv(path,skiprows=1)
Data = Data.drop('!6',axis=1)
values_to_keep =  ["VALN", "ZCB", "EQ", "RE"]
#Interest rate treatment : keep ZC with duration
Duration = 10 
Data =  Data[Data['CLASS'].isin(values_to_keep)]
Data = Data[~((Data['CLASS'] == 'ZCB') & (Data['TERM'] != Duration))]
Data.drop('TERM',axis=1) #Remove term , not needed anymore



In [ ]:
df = Data

# Fondre le dataframe pour le mettre sous forme longue
df_melted = df.melt(id_vars=['SIMULATION', 'CLASS', 'MEASURE'], 
                    value_vars=[col for col in df.columns if col not in ['SIMULATION', 'CLASS', 'MEASURE']], 
                    var_name='Date', 
                    value_name='Value')

# Créer une nouvelle colonne pour la combinaison de la classe et de la date
df_melted['Feature'] = df_melted['CLASS'] + '_' + df_melted['MEASURE'] + '_' + df_melted['Date']

# Pivoter le dataframe pour que les colonnes soient les nouvelles caractéristiques
df_pivot = df_melted.pivot_table(index='SIMULATION', columns='Feature', values='Value')

# Réinitialiser l'index
df_pivot.reset_index(inplace=True)

print(df_pivot.columns)